In [369]:
import pandas as pd

In [370]:
df_bordeaux = pd.read_csv("CSV/bordeaux/listings_bordeaux.csv")


In [371]:
df_bordeaux.groupby("neighbourhood_cleansed").apply(lambda s: pd.Series({ 
    "host count": s["host_id"].nunique(), 
    "sum reviews": s["number_of_reviews"].sum(), 
}))

,host count,sum reviews
neighbourhood_cleansed,,
3M-Bourgailh,8,98
Ambars-et-Lagrave,65,904
Ambs,49,1714
Arago-La Chataigneraie,43,1583
Arlac,90,2973
...,...,...
Sardine,17,788
Talence,429,13571
Toctoucau,16,479


In [372]:
df_bordeaux[['id',"host_verifications"]].groupby("host_verifications").count()

,id
host_verifications,
"['email', 'phone', 'work_email']",677
"['email', 'phone']",9287
"['email', 'work_email']",1
['email'],45
"['phone', 'work_email']",24
['phone'],820
[],31


In [373]:
pd.pivot_table(df_bordeaux, index="host_verifications", values='id', aggfunc="count")

,id
host_verifications,
"['email', 'phone', 'work_email']",677
"['email', 'phone']",9287
"['email', 'work_email']",1
['email'],45
"['phone', 'work_email']",24
['phone'],820
[],31


In [374]:
df_updated = df_bordeaux.replace(to_replace ='work_email', value = 'work', regex = True)

In [375]:
pd.pivot_table(df_updated, index="host_verifications", values='id', aggfunc="count")

,id
host_verifications,
"['email', 'phone', 'work']",677
"['email', 'phone']",9287
"['email', 'work']",1
['email'],45
"['phone', 'work']",24
['phone'],820
[],31


In [376]:
import numpy as np
# pd.pivot_table(df_updated, index="room_type", values='amenities', aggfunc=["mean", np.std])

df_updated[["id","room_type"]].groupby('room_type').agg(["mean","std"])

id              
                         mean           std
room_type                                  
Entire home/apt  1.239859e+17  2.520322e+17
Hotel room       3.346399e+07  4.774587e+06
Private room     1.054322e+17  2.378023e+17
Shared room      1.101446e+17  2.402290e+17

In [377]:
import numpy as np
for i in df_bordeaux['host_verifications'].unique():
    if i == np.empty([1,1]):
        print("Ok")

/tmp/ipykernel_54717/3367585850.py:3: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if i == np.empty([1,1]):


In [378]:
type(df_bordeaux['host_verifications'].unique())

numpy.ndarray

In [379]:
df_bordeaux["np_amenities"] =  df_bordeaux['amenities'].apply(len)
df_bordeaux[["np_amenities","room_type"]].groupby('room_type').agg(["mean","std"])

np_amenities            
                        mean         std
room_type                               
Entire home/apt   471.416888  284.652262
Hotel room        451.288889  298.016292
Private room      361.963512  242.113035
Shared room       251.602941  160.739907

In [380]:
df_bordeaux["price"] = df_bordeaux["price"].str.replace('$','')
df_bordeaux["price"] = df_bordeaux["price"].str.replace(',','').astype(float)
max(df_bordeaux["price"])

/tmp/ipykernel_54717/3438453612.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_bordeaux["price"] = df_bordeaux["price"].str.replace('$','')


8890.0

In [381]:
df_bordeaux["bathrooms_text"].unique()

array(['1.5 baths', '3 baths', '1 bath', '1 private bath', nan, '2 baths',
       '1 shared bath', 'Shared half-bath', '4 baths', '8 baths',
       '5 baths', '4.5 baths', '2.5 baths', '0 baths', '1.5 shared baths',
       '3.5 baths', 'Half-bath', '3 shared baths', '6 baths', '5.5 baths',
       '7 baths', '0 shared baths', '2.5 shared baths',
       'Private half-bath', '2 shared baths', '6.5 baths',
       '4 shared baths', '11.5 baths', '7.5 baths'], dtype=object)

In [382]:
df_bordeaux[['N', 'bath']] = df_bordeaux['bathrooms_text'].str.split(' ', 1, expand=True)

/tmp/ipykernel_54717/3056524629.py:1: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df_bordeaux[['N', 'bath']] = df_bordeaux['bathrooms_text'].str.split(' ', 1, expand=True)


In [384]:
import re

def first(s):
    return(re.findall('(\d+\.\d+)', s))

def last(s):
    return(re.split('(\d+)', s)[-1])

df_bordeaux['N'] = df_bordeaux['bathrooms_text'].apply(lambda x :  first(str(x)))
df_bordeaux['bath'] = df_bordeaux['bathrooms_text'].apply(lambda x : last(str(x)))
df_bordeaux['N'] = pd.DataFrame([x for x in df_bordeaux['N'] ]) # remove list from N

In [388]:
pd.pivot_table(df_bordeaux, index='N', values='id', aggfunc="count")

,id
N,
1.5,546
11.5,1
2.5,168
3.5,31
4.5,6
5.5,5
6.5,3
7.5,1
